# IKEv2 responder test

In [15]:
!apt update && apt install -y libpcap-dev
%pip install --pre scapy[basic]
%pip install pycryptodomex

Get:1 http://deb.debian.org/debian bullseye InRelease [116 kB]
Get:2 http://security.debian.org/debian-security bullseye-security InRelease [44.1 kB]3m
Get:3 http://deb.debian.org/debian bullseye-updates InRelease [39.4 kB]
Get:4 http://deb.debian.org/debian bullseye/main amd64 Packages [8182 kB]
Get:5 http://security.debian.org/debian-security bullseye-security/main amd64 Packages [153 kB]
Get:6 http://deb.debian.org/debian bullseye-updates/main amd64 Packages [2592 B]m
Fetched 8537 kB in 2s (5008 kB/s)3m                       
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
All packages are up to date.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libpcap-dev is already the newest version (1.10.0-2).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.
Note: you may need to res

In [1]:
import socket
hostip = socket.gethostbyname(socket.gethostname())
print("IP address of host:", hostip)

IP address of host: 172.17.0.2


In [2]:
from scapy.all import *
from scapy.contrib.ikev2 import *
from scapy.utils import inet_aton
import binascii
import os
import ikev2_lib
import socket

/usr/local/lib/python3.10/site-packages/scapy/layers/ipsec.py:471: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
/usr/local/lib/python3.10/site-packages/scapy/layers/ipsec.py:485: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


IKEv2 specification: https://tools.ietf.org/pdf/rfc7296.pdf

In [3]:
INITIATOR_IP = '172.17.0.3'
INITIATOR_PORT = 500
RESPONDER_IP = '172.17.0.2'
RESPONDER_PORT = 500

### Create dummy UDP listener to prevent ICMP port unreachable messages

In [4]:
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock.bind((RESPONDER_IP, RESPONDER_PORT))

### Initialise local parameters and secrets

In [5]:
nonce_r = os.urandom(32)
spi_r = b'Ysblokje'

dh = ikev2_lib.DiffieHellman()

In [6]:
capture = sniff(filter=f"dst host {RESPONDER_IP} and dst port {RESPONDER_PORT}", count=1)
capture.summary()

Ether / IP / UDP 172.17.0.3:isakmp > 172.17.0.2:isakmp / IKEv2 / IKEv2_payload_SA / IKEv2_payload_KE / IKEv2_payload_Nonce / IKEv2_payload_Notify / IKEv2_payload_Notify / IKEv2_payload_Notify / IKEv2_payload_Notify / IKEv2_payload_Notify


In [7]:
packet = capture[0][IP]
packet.show()

###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 492
  id        = 122
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = udp
  chksum    = 0xe05f
  src       = 172.17.0.3
  dst       = 172.17.0.2
  \options   \
###[ UDP ]### 
     sport     = isakmp
     dport     = isakmp
     len       = 472
     chksum    = 0x5a11
###[ IKEv2 ]### 
        init_SPI  = '\x04\\x90\\x94(\x14u\\x9c\\xe3'
        resp_SPI  = ''
        next_payload= SA
        version   = 0x20
        exch_type = IKE_SA_INIT
        flags     = Initiator
        id        = 0
        length    = 464
###[ IKEv2 SA ]### 
           next_payload= KE
           res       = 0
           length    = 48
           \prop      \
            |###[ IKEv2 Proposal ]### 
            |  next_payload= last
            |  res       = 0
            |  length    = 44
            |  proposal  = 1
            |  proto     = IKEv2
            |  SPIsize   = 0
            |  trans_nb  = 4
            

In [8]:
dh_a = dh.generate_public().to_bytes(length=256, byteorder='big') 
dh_b = int.from_bytes(packet[IKEv2_payload_KE].load, byteorder='big')

dhs = dh.generate_shared(dh_b).to_bytes(length=256, byteorder='big')
print(f"shared Diffie Hellman secret: {binascii.b2a_hex(dhs).decode()}")

shared Diffie Hellman secret: b83d180123b5fa2c6a7c19644c61ea853753266f7f49ee48710d481ee952eda6923ea8e9bd10527a89d8bbabb92e347c2c489ca395d6f825bd96ea95313dc72e73472c234319c5c973c9d62d491b87aa66f9c26051ec02d81b88a00dcfc50c544aa16dd136fef93a6bc0c1c571a3852b7b8ddc69985de04bc9316db2c87ddfc6b06e3aade594f4317fd3feb26ed8dd51f6bcad836257158e5345e4323da3cdc88409a8378ed3e9aa34d68377b40b7510710c8c4622127e156572baa0785d353d641a767b1a735cc2e942b1ad91052683f63208c240a02945e50c49fc13efc7e36e6fa025b9d833d7765dd63d68c8d7f05797cd6c9bd9946ffcc5794da2e27c99


In [9]:
nonce_i = packet[IKEv2_payload_Nonce].load
spi_i = packet[IKEv2].init_SPI

print(f"SPIi: {spi_i}")
print(f"SPIr: {spi_r}")

print(f"Ni: {binascii.b2a_hex(nonce_i).decode()}")
print(f"Nr: {binascii.b2a_hex(nonce_r).decode()}")

SPIi: b'\x04\x90\x94(\x14u\x9c\xe3'
SPIr: b'Ysblokje'
Ni: a0c23beaa436a6f6f09da8d3fef99db7e9e3f44f3ffe83905731eaf9e882df5c
Nr: f2d748145dbcad2ca3f58c72fe8adbc6d7738c6b40aa4f3d9ccd0da62357d3ca


### Calculate SKEYSEED and derive keys ([RFC7296 2.14](https://datatracker.ietf.org/doc/html/rfc7296#section-2.14))

In [10]:
skeyseed = ikev2_lib.PrfHmacSha256(nonce_i + nonce_r, dhs)

print(f"SKEYSEED: {binascii.b2a_hex(skeyseed).decode()}")

SKEYSEED: 898bb94b7370a6716991e5a01cefb36605ef3620a692d5c5621d1c7b6cf45fc6


Keys needed: SK_d | SK_ai | SK_ar | SK_ei | SK_er | SK_pi | SK_pr
* Sk_d is 32 bytes (256 bits) based on PRF_HMAC_SHA2_256
* Sk_ai is 20 bytes (160 bits) based on AUTH_HMAC_SHA1_96
* Sk_ar is 20 bytes (160 bits) based on AUTH_HMAC_SHA1_96
* Sk_ei is 32 bytes (256 bits) based on ENCR_AES_CBC
* Sk_er is 32 bytes (256 bits) based on ENCR_AES_CBC
* Sk_pi is 32 bytes (256 bits) based on PRF_HMAC_SHA2_256
* Sk_pr is 32 bytes (256 bits) based on PRF_HMAC_SHA2_256

Total is **200 bytes**

In [11]:
prfplusoutput = ikev2_lib.PrfPlus(ikev2_lib.PrfHmacSha256, skeyseed, nonce_i + nonce_r + spi_i + spi_r, 200)
#print(f"prf+ output: {binascii.b2a_hex(prfplusoutput).decode()}")
sk_d = prfplusoutput[:32]
sk_ai = prfplusoutput[32:52]
sk_ar = prfplusoutput[52:72]
sk_ei = prfplusoutput[72:104]
sk_er = prfplusoutput[104:136]
sk_pi = prfplusoutput[136:168]
sk_pr = prfplusoutput[168:200]
print(f"Sk_d: {binascii.b2a_hex(sk_d).decode()}")
print(f"Sk_ai: {binascii.b2a_hex(sk_ai).decode()}")
print(f"Sk_ar: {binascii.b2a_hex(sk_ar).decode()}")
print(f"Sk_ei: {binascii.b2a_hex(sk_ei).decode()}")
print(f"Sk_er: {binascii.b2a_hex(sk_er).decode()}")
print(f"Sk_pi: {binascii.b2a_hex(sk_pi).decode()}")
print(f"Sk_pr: {binascii.b2a_hex(sk_pr).decode()}")

Sk_d: 7b72f4f44f741541ffb52135d134560474cc9c8bc8e3555ede57ed9ce2b91cf0
Sk_ai: cd47ce2b53cbd8183d43cae78aeb54828c32d85f
Sk_ar: 2278a0153db3c34f70dd7d8857db15b6269699a0
Sk_ei: fcd5c2e892c47d7184c010d421a7606d136ac8d3e108c6ea51aaca8a43a1c313
Sk_er: e6b50a6d4410f2d2814b3b39815fa9a0918daeee9cdccf71a5666c6531e3498a
Sk_pi: dc2731d60465079ada86dd27c69da50436d92ce0186caae5cbe71ab1408f7164
Sk_pr: 8caa8599bcc12c3453f49ec32366c8d005c3e84df770f8c6eb73f61b11a36f3e


### Build IKE_SA_INIT response

In [12]:
# Get proposal from Initiator IKE_SA_INIT 
proposal = packet[IKEv2_payload_Proposal]

hdr = IKEv2(init_SPI = spi_i, resp_SPI=spi_r, next_payload = 'SA', exch_type = 'IKE_SA_INIT', flags='Response')
sa = IKEv2_payload_SA(next_payload = 'KE', prop=proposal)
ke = IKEv2_payload_KE(next_payload = 'Nonce', group = '2048MODPgr', load = dh_a)
nonce = IKEv2_payload_Nonce(next_payload = 'None', load = nonce_r)

ike_sa_init = hdr/sa/ke/nonce

packet = IP(dst = INITIATOR_IP)/UDP(dport = INITIATOR_PORT, sport = RESPONDER_PORT)/ike_sa_init

#packet.show()


### Send IKE_SA_INIT response and receive IKE_AUTH

In [13]:
ans = sr1(packet)
ans.show()

Begin emission:
Finished sending 1 packets.
..*
Received 3 packets, got 1 answers, remaining 0 packets
###[ IP ]### 
  version   = 4
  ihl       = 5
  tos       = 0x0
  len       = 392
  id        = 8640
  flags     = DF
  frag      = 0
  ttl       = 64
  proto     = udp
  chksum    = 0xbf7d
  src       = 172.17.0.3
  dst       = 172.17.0.2
  \options   \
###[ UDP ]### 
     sport     = isakmp
     dport     = isakmp
     len       = 372
     chksum    = 0x59ad
###[ IKEv2 ]### 
        init_SPI  = '\x04\\x90\\x94(\x14u\\x9c\\xe3'
        resp_SPI  = 'Ysblokje'
        next_payload= Encrypted
        version   = 0x20
        exch_type = IKE_AUTH
        flags     = Initiator
        id        = 1
        length    = 364
###[ IKEv2 Encrypted and Authenticated ]### 
           next_payload= IDi
           res       = 0
           length    = 336
           load      = '\\xa3\\xd3#\\xab7pL\\xf0\\xda\x1a\x15\x10[/\\xda\\xe5\x0bI<C\\x8b\\xbb\\xf7\\x8a\\x8d\\xec:\x07\\xd3쇧\x11\\xb8\\xdb.\\xa8

### Verify checksum and decrypt payload ([RFC7296 3.14](https://datatracker.ietf.org/doc/html/rfc7296#section-3.14))

In [30]:
from Cryptodome.Cipher import AES
import hmac, hashlib
cipher_block_size = 16   # AES_CBC
integrity_hash_size = 12 # HMAC_SHA1_96

auth_data = raw(ans[IKEv2])[:-integrity_hash_size]
iv = ans[IKEv2_payload_Encrypted].load[:cipher_block_size]
encrypted = ans[IKEv2_payload_Encrypted].load[cipher_block_size:-integrity_hash_size]
checksum = ans[IKEv2_payload_Encrypted].load[-integrity_hash_size:]

checksum_calculated = hmac.new(sk_ai, auth_data, hashlib.sha1).digest()[:integrity_hash_size]

print(f"IV: {binascii.b2a_hex(iv).decode()}")
print(f"Checksum in packet: \t{binascii.b2a_hex(checksum).decode()}")
print(f"Calculated Checksum: \t{binascii.b2a_hex(checksum_calculated).decode()}\n")

plain_padded = AES.new(sk_ei, AES.MODE_CBC, iv).decrypt(encrypted)
pad_length = plain_padded[-1]
plain = plain_padded[:-pad_length-1]
payload = IKEv2_payload_IDi(plain) # Cast decrypted payload to IKEv2 payload(s), for now assumes IDi payload
payload.show()

IV: a3d323ab37704cf0da1a15105b2fdae5
Checksum in packet: 	74ecb4c20b3dda2e68719fc4
Calculated Checksum: 	74ecb4c20b3dda2e68719fc4

###[ IKEv2 Identification - Initiator ]### 
  next_payload= Notify
  res       = 0
  length    = 12
  IDtype    = IPv4_addr
  ProtoID   = Unused
  Port      = Unused
  load      = '\\xac\x11\x00\x03'
###[ IKEv2 Notify ]### 
     next_payload= IDr
     res       = 0
     length    = 8
     proto     = Reserved
     SPIsize   = 0
     type      = INITIAL_CONTACT
     SPI       = ''
     load      = ''
###[ IKEv2 Identification - Responder ]### 
        next_payload= AUTH
        res       = 0
        length    = 12
        IDtype    = IPv4_addr
        ProtoID   = Unused
        Port      = Unused
        load      = '\\xac\x11\x00\x02'
###[ IKEv2 Authentication ]### 
           next_payload= SA
           res       = 0
           length    = 40
           auth_type = Shared Key Message Integrity Code
           res2      = 0x0
           load      = 'ȕ\\xcb\